In [2]:
# import necessary imports
import numpy as np

In [2]:
# data preprocessing 
    # load encoded train/validation/test sets
    # chunken und batchen

batch_size = 1
chunk_size = 1

In [60]:
# forward pass
# token embedding table, outputs: logits
class neural_embedding:
    def __init__(self, vocab_size):
        # they call nn.Embedding, need to look up what that is, exactly
        self.token_embedding_table = np.random.rand(vocab_size, vocab_size)

    def calculate_softmax(self, x):
        """Takes input array x and returns softmax."""
        soft_x = np.exp(x - np.max(x))
        softer_x = soft_x / np.sum(soft_x)
        return softer_x

    def calculate_cross_entropy(self, y_hatless, y_hat):
        """
        Takes target (y_hatless) and prediction (y_hat) and computes cross entropy loss.
        """
        _, _, vocab_size = y_hat.shape
        #print(f"size before: {y_hatless.shape}")
        
        y_hat = y_hat.reshape(y_hat.shape[0]*y_hat.shape[1], y_hat.shape[2])
        y_hatless_flat = y_hatless.reshape(-1)
        y_hatless_hot = np.eye(vocab_size)[y_hatless_flat]
        print(y_hatless_hot)

        #y_hatless = y_hatless.reshape(y_hatless.shape[0]*y_hatless.shape[1], y_hatless.shape[2])
        #print(f"size after: {y_hatless.shape}")
        
        y_hat = self.calculate_softmax(y_hat)
        
    
        return -np.sum(y_hatless_hot*np.log(y_hat))

        #-np.sum(y * np.log(y_pred + eps))
    
    def forward(self, idx, targets=None):
        # idx and targets both (B,T) tensor of integers
        batch_size, chunk_size = idx.shape
        #print(f"Shape: {idx.shape}, batches: {batch_size}, chunks: {chunk_size}")
        #print(f"Vocab size, hopefully: {self.token_embedding_table[0].size}")
        logits = np.zeros((batch_size, chunk_size, (self.token_embedding_table[0].size)))
        

        def logitte(batch_size, chunk_size, input):
            for batch in range(batch_size):
                for chunk in range(chunk_size):
                    # (B,T,C) b=batch_size, t="time"=chunk_size, c=vocab_size
                    logits[batch][chunk] = self.token_embedding_table[input[batch][chunk]]
                    
            #print(f"Shape: {logits.shape}, \n logits: {logits}")
            return logits

        input_logits = logitte(batch_size, chunk_size, idx)
        
        if targets is not None:
            loss = self.calculate_cross_entropy(targets, input_logits)

            return input_logits, loss
            

        return input_logits
    
    def backward():
        ...

In [61]:
vocab_size = 80

my_neural_embedding = neural_embedding(vocab_size)
#print(my_neural_embedding.token_embedding_table)
uglies_array_youve_ever_seeeeeen = np.array([[1,2,3], [3,4,7]])
my_input_logits, my_loss = my_neural_embedding.forward(uglies_array_youve_ever_seeeeeen, uglies_array_youve_ever_seeeeeen)
print(f"loss: {my_loss}")


[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0.